# Import libraries and load data

In [1]:
# Standard Libraries #
import pandas as pd
#import numpy as np
import datetime
from datetime import timedelta
import pathlib
import h5py

# Custom functions #
from Tools import GetNeuropaceCatalog, GetChannelAxis, GetPeakFrequencies, GetWaveletsArray, GetTimeAxis, TabulateWavelets, GetArtifactTags, format_time

pd.set_option('display.max_rows', 100) #set max of display rows when printing dataframe

In [11]:
# User-specified inputs #
patient_id = 'PR05'
input_dir = f'/userdata/dastudillo/patient_data/stage2/{patient_id}' #surveys csv (from redcap) and rns catalog (from box) should be located in the same directory 
output_dir = f'/userdata/dastudillo/patient_data/stage2/{patient_id}'
surveys_raw = pd.read_csv(pathlib.Path(input_dir, f'{patient_id}_Stage2Surveys.csv'))
rns_raw = GetNeuropaceCatalog(patient_id)

magnet_on    = True
scheduled_on = False
realtime_on  = False

start, stop = '2024-09-06', '2024-09-07'

In [12]:
# Clean up and extract relevant columns from rns catalog # 
# Do not modify
rns_raw = rns_raw.rename(columns={'ECoG trigger':'TriggerType',
                                  'Timestamp':'Start_Timestamp_PT',
                                  'Raw local timestamp':'Trigger_Timestamp_Local',
                                  'ECoG length':'Total_Duration',
                                  'ECoG pre-trigger length':'PreTrigger_Duration'})

if (magnet_on==True) & (scheduled_on==False) & (realtime_on==False):
    mask = (rns_raw.TriggerType=='Magnet')
if (magnet_on==True) & (scheduled_on==True) & (realtime_on==False):
    mask = ((rns_raw.TriggerType=='Magnet') ^ (rns_raw.TriggerType=='Scheduled'))
if (magnet_on==False) & (scheduled_on==False) & (realtime_on==True):
    mask = (rns_raw.TriggerType=='Real_Time')

rns_clean = rns_raw.loc[mask, ['Filename', 'TriggerType', 'Trigger_Timestamp_Local', 'PreTrigger_Duration', 'Total_Duration']].reset_index(drop=True)
rns_clean['Trigger_Timestamp_Local'] = pd.to_datetime(rns_clean['Trigger_Timestamp_Local'], format='%Y-%m-%d %H:%M:%S.%f')

start_timestamp_local = []
for i, j in zip(rns_clean.Trigger_Timestamp_Local, rns_clean.PreTrigger_Duration):
    start_timestamp_local.append(i - timedelta(seconds=j))
rns_clean.insert(2, 'Start_Timestamp_Local', start_timestamp_local)

stop_timestamp_local = []
for i, j in zip(rns_clean.Start_Timestamp_Local, rns_clean.Total_Duration):
    stop_timestamp_local.append(i + timedelta(seconds=j))
rns_clean.insert(3, 'Stop_Timestamp_Local', stop_timestamp_local)

In [13]:
# Read HDF5 file containing preprocessed data #
PREPROC_DIR = f'/data_store0/presidio/NeuroPace_DB/{patient_id}/NP_PROC_HDF/signals_db-pipeline_d8bc5025145576d05d1934d0f027bbb1.hdf5'
PREPROC_DATA = h5py.File(PREPROC_DIR, 'r')

In [14]:
#filter files according to date range specified in user inputs. 
mask = (rns_clean.Trigger_Timestamp_Local>=datetime.datetime.strptime(start, '%Y-%m-%d')) & (rns_clean.Trigger_Timestamp_Local<=datetime.datetime.strptime(stop, '%Y-%m-%d'))
rns_filter = rns_clean.loc[mask].reset_index(drop=True)

In [15]:
# Get list of file names and start timestamps of recordings
files       = rns_filter.Filename
files_start = rns_filter.Start_Timestamp_Local
files_triggertype = rns_filter.TriggerType
files_triggertimestamp = rns_filter.Trigger_Timestamp_Local
print('Number of files: ', len(files))

Number of files:  2


# Tabulate preprocessed data

In [16]:
#Use custom functions to get wavelets tabulated
#NOTE: Suuuper time consuming, don't use to tabulate large number of files (max 5), warning: huge dataframe!!
#TO DO: adapt script to send as server job

allfiles_df = pd.DataFrame()

for i in range(len(files)):
    channels = GetChannelAxis(PREPROC_DATA[files[i]])
    peak_freqs = GetPeakFrequencies(PREPROC_DATA[files[i]])
    wavelets = GetWaveletsArray(PREPROC_DATA[files[i]])
    n_samples = wavelets.shape[2]
    time_axis = GetTimeAxis(PREPROC_DATA[files[i]], n_samples, files_start[i])

    artifact_tags = GetArtifactTags(PREPROC_DATA[files[i]], files_start[i])
    artifact_tags['ArtifactTimestamp'] = pd.to_datetime(pd.Series(format_time(artifact_tags.ArtifactTimestamp)))

    wavelets_ch1 = TabulateWavelets(channels[0], wavelets[0], peak_freqs, time_axis)
    wavelets_ch2 = TabulateWavelets(channels[1], wavelets[1], peak_freqs, time_axis)
    wavelets_ch3 = TabulateWavelets(channels[2], wavelets[2], peak_freqs, time_axis)
    wavelets_ch4 = TabulateWavelets(channels[3], wavelets[3], peak_freqs, time_axis)
    all_wavelets = pd.concat([wavelets_ch1, wavelets_ch2, wavelets_ch3, wavelets_ch4], axis=1).T.drop_duplicates().T
    all_wavelets['Timestamp'] = pd.to_datetime(all_wavelets['Timestamp'])
    del wavelets_ch1, wavelets_ch2, wavelets_ch3, wavelets_ch4
    del wavelets

    file_df = all_wavelets.merge(artifact_tags, how='inner', left_on='Timestamp', right_on='ArtifactTimestamp')
    file_df['Filename'] = files[i]
    file_df['TriggerType'] = files_triggertype[i]
    file_df['TriggerLocalTimestamp'] = files_triggertimestamp[i]
    
    col0 = file_df.pop('Filename')
    col1 = file_df.pop('TriggerType')
    col2 = file_df.pop('TriggerLocalTimestamp')
    
    file_df.insert(0, 'Filename', col0)
    file_df.insert(1, 'TriggerType', col1)
    file_df.insert(2, 'TriggerLocalTimestamp', col2)
    
    del artifact_tags
    print(f'Adding file {i+1} to final dataframe')
    print('')
    
    allfiles_df = pd.concat([allfiles_df, file_df], ignore_index=True)

PREPROC_DATA.close() #close hdf5 file

Adding file 1 to final dataframe

Adding file 2 to final dataframe



In [17]:
allfiles_df

,Filename,TriggerType,TriggerLocalTimestamp,Timestamp,PeakFrequency,Amyg1 - Amyg2,Amyg3 - Amyg4,VC1 - VC2,VC3 - VC4,ArtifactTimestamp,ArtifactTag
0,133701433372840000.dat,Magnet,2024-09-06 01:19:33.238,2024-09-06 01:16:53.266,120.0,3.682879,0.906325,1.005418,1.810948,2024-09-06 01:16:53.266,0.0
1,133701433372840000.dat,Magnet,2024-09-06 01:19:33.238,2024-09-06 01:16:53.314,120.0,2.643185,1.054987,1.294203,2.525001,2024-09-06 01:16:53.314,0.0
2,133701433372840000.dat,Magnet,2024-09-06 01:19:33.238,2024-09-06 01:16:53.362,120.0,1.741089,0.505124,1.735792,1.033904,2024-09-06 01:16:53.362,0.0
3,133701433372840000.dat,Magnet,2024-09-06 01:19:33.238,2024-09-06 01:16:53.410,120.0,2.340128,0.636996,0.804513,1.666675,2024-09-06 01:16:53.410,0.0
4,133701433372840000.dat,Magnet,2024-09-06 01:19:33.238,2024-09-06 01:16:53.458,120.0,1.558092,2.002311,2.183577,1.800471,2024-09-06 01:16:53.458,0.0
...,...,...,...,...,...,...,...,...,...,...,...
399875,133701683420930000.dat,Magnet,2024-09-06 19:37:09.831,2024-09-06 19:38:29.543,1.0,12.149955,11.931016,26.478907,7.782563,2024-09-06 19:38:29.543,0.0
399876,133701683420930000.dat,Magnet,2024-09-06 19:37:09.831,2024-09-06 19:38:29.591,1.0,12.325196,11.908784,26.520653,7.696458,2024-09-06 19:38:29.591,0.0
399877,133701683420930000.dat,Magnet,2024-09-06 19:37:09.831,2024-09-06 19:38:29.639,1.0,12.496888,11.882606,26.562113,7.616193,2024-09-06 19:38:29.639,0.0
399878,133701683420930000.dat,Magnet,2024-09-06 19:37:09.831,2024-09-06 19:38:29.687,1.0,12.671425,11.860647,26.605243,7.547533,2024-09-06 19:38:29.687,0.0


In [ ]:
allfiles_df.to_csv(pathlib.Path(output_dir, f'{patient_id}_Stage2_wavelets_{start}_{stop}.csv'), index=False)
del allfiles_df